# Summarization via Langchain Map Reduce

This notebook walks through how to use LangChain and Azure document intelligence for summarization over a list of documents. It covers three different chain types: `stuff`, `map_reduce`, and `refine`. For a more in depth explanation of what these chain types are, see [here](https://docs.langchain.com/docs/components/chains/index_related_chains).

### Loading the Keys

In [1]:
import os
from dotenv import load_dotenv
load_dotenv('azure.env',override = True)

True

### Importing the libraries

In [2]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.schema import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.schema.runnable import RunnableMap
from langchain_core.runnables import RunnableConfig
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

from azure.ai.documentintelligence.models import DocumentAnalysisFeature
from langchain_community.document_loaders.doc_intelligence import AzureAIDocumentIntelligenceLoader

In [3]:


# from langchain.memory import ConversationBufferMemory
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# # from langchain.memory import PostgresChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
# from langchain_core.runnables import RunnableLambda, RunnablePassthrough


### Uploading the document through Azure Document Intelligence

In [4]:
api_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")
api_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
# print(f"API Key: {api_key}, Type: {type(api_key)}")
# print(f"API Endpoint: {api_endpoint}, Type: {type(api_endpoint)}")

## Replace the file path with your local file path 

In [5]:
file1 = r'C:\Users\nag\Documents\Microsoft\Customer data\Infosys_Legal_docs_summary\Health System Provider Agreement.pdf'

In [6]:
file2 = r'C:\Users\nag\Documents\Microsoft\Customer data\Infosys_Legal_docs_summary\CAContract with DHS & CMS.pdf'

In [7]:
file3 = r'C:\Users\nag\Documents\Microsoft\Customer data\Infosys_Legal_docs_summary\Group-Healthcare-Contract.pdf'

In [8]:
file4 = r'C:\Users\nag\Documents\Microsoft\Customer data\Infosys_Legal_docs_summary\HealthCare Service Provider Agreement.pdf'

In [9]:
file5 = r'C:\Users\nag\Documents\Microsoft\Customer data\Infosys_Legal_docs_summary\HealthCare Service Provider Agreement.pdf'

In [10]:
file6 = r'C:\Users\nag\Documents\Microsoft\Customer data\Infosys_Legal_docs_summary\Oregon Health Authority - Coordinated Care Organization.pdf'

In [11]:
loader = AzureAIDocumentIntelligenceLoader(file_path=file1, 
                                           api_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY"), 
                                           api_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT"),
                                           api_model="prebuilt-layout",
                                           api_version="2024-02-29-preview",
                                           mode='markdown',
                                           analysis_features = [DocumentAnalysisFeature.OCR_HIGH_RESOLUTION])
docs = loader.load()

In [12]:
from IPython.display import display, Markdown, Latex
display(Markdown(docs[-1].page_content))

AmeriHealth Caritas Ohio, Inc. PROVIDER SERVICES AGREEMENT

with

[HEALTH SYSTEM]

AmeriHealth Caritas Ohio, Inc. PROVIDER SERVICES AGREEMENT
===

This Provider Services Agreement (the "Agreement"), dated as of the Effective Date (defined below), is made by and between AmeriHealth Caritas Ohio, Inc., a corporation organized under the laws of the State of Ohio, (hereinafter referred to as "ACOH") and [HEALTH SYSTEM NAME] ("Health System"), contracting for itself and on behalf of its following affiliated providers: (List them here), each of which is referred to as a "Provider" or "Health System Provider" and all of which are collectively referred to as the "Health System Providers" under this Agreement.

WHEREAS, ACOH is a managed care organization that is responsible for providing or arranging for the provision of health care services to its Members; and

WHEREAS, Health System has entered into an agreement with or is otherwise affiliated with the Health System Providers, and has full legal authority to enter into this Agreement on their behalf; and

WHEREAS, the Health System Providers are duly licensed and otherwise qualified to furnish certain health care services; and

WHEREAS, Health System and ACOH mutually desire to enter into this Agreement, whereby the Health System Providers shall render services to Members enrolled with ACOH and be compensated by ACOH in accordance with the terms and conditions hereof.

NOW, THEREFORE, in consideration of the mutual promises made herein, it is mutually agreed by and between ACOH and Health System as follows:


## 1\. DEFINITIONS

As used in this Agreement, each of the following terms shall have the meaning specified herein, unless the context clearly requires otherwise.

1.1 AFFILIATES. An Affiliate is any corporation or other organization that is identified as an Affiliate in a written notice to Health System and is owned or controlled, either directly or through parent or subsidiary corporations, by or under common control with, ACOH. ACOH shall give Health System thirty (30) days advance written notice of the addition of Affiliates added under this provision. Unless otherwise specified in this Agreement or any other attachment hereto, references to "ACOH" shall include the Affiliates referenced in Appendix C.

1.2 AGENCY. The State and/or Federal governmental agency that administers the Program(s) under which ACOH is obligated to provide or arrange for the provision of Covered Services.

1.3 AGENCY CONTRACT. The contract or contracts between ACOH and the Agency, as in effect from time to time, pursuant to which ACOH is responsible for coordinating health care services and supplies for Program recipients enrolled with ACOH.

1.4 CLEAN CLAIM. A claim for payment for a health care service, which has been received by ACOH, has no defect or impropriety. A defect or impropriety shall include a lack of required substantiating documentation or a particular circumstance requiring special treatment that prevents timely payment from being made on the claim. Consistent with 42 CFR §447.45(b), the term shall

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="1" -->

not include a claim from a health care provider who is under investigation for fraud or abuse regarding that claim, or a claim under review for medical necessity.

1.5 COVERED SERVICES. Those Medically Necessary health care services and supplies to which Members are entitled pursuant to the Agency Contract, and which shall be provided to Members by Health System, as described more specifically in Appendix A. Covered Services shall be furnished in the amount, duration and scope required under the Program. Without limiting the foregoing, Covered Services may be furnished through telehealth (as defined in OAC 5160-1-18 (the "Ohio Telehealth Rule")), subject to all applicable requirements of the Ohio Telehealth Rule being met, including but not limited to requirements for provider type.

1.6 EFFECTIVE DATE. The later of (i) the effective date on the signature page of this Agreement or (ii) the effective date of the Agency Contract, provided that the Health System Providers have been successfully credentialed and that all required regulatory approvals have been obtained by ACOH.

1.7 EMERGENCY MEDICAL CONDITION. Health care services provided to a Member after the sudden onset of a medical condition that manifests itself by acute symptoms of sufficient severity or severe pain, such that a prudent layperson, who possesses an average knowledge of health and medicine, could reasonably expect the absence of immediate medical attention to result in:

(a) Placing the health of the Member (or with respect to a pregnant woman, the health of the Member or her unborn child) in serious jeopardy;

(b) Serious impairment to bodily functions; or

(c) Serious dysfunction of any bodily organ or part.

1.8 EMERGENCY SERVICES. Covered inpatient and outpatient services that are furnished by a provider that is qualified to furnish these services under 42 C.F.R. Section 438.114(a) and 42 U.S.C. Section 1932(b)(2) and that are needed to screen, evaluate, and stabilize an Emergency Medical Condition.

1.9 GROUP PHYSICIAN. A physician who practices with a Health System Provider that is a group practice as an employee, partner, shareholder, or contractor.

1.10 MEDICALLY NECESSARY. "Medical Necessity" is defined in accordance with OAC §5160-1- 01, which provides as follows:

(a) Medical necessity for individuals covered by early and periodic screening, diagnosis and treatment (EPSDT) is defined as procedures, items, or services that prevent, diagnose, evaluate, correct, ameliorate, or treat an adverse health condition such as an illness, injury, disease or its symptoms, emotional or behavioral dysfunction, intellectual deficit, cognitive impairment, or developmental disability.

(b) Medical necessity for individuals not covered by EPSDT is defined as procedures, items, or services that prevent, diagnose, evaluate, or treat an adverse health condition such as an illness, injury, disease or its symptoms, emotional or behavioral dysfunction, intellectual deficit, cognitive impairment, or developmental disability and without which the person can be expected to suffer prolonged, increased or new morbidity; impairment of function; dysfunction of a body organ or part; or significant pain and discomfort.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="2" -->

(c) Conditions of medical necessity are met if all the following apply: (i) meets generally accepted standards of medical practice; (ii) clinically appropriate in its type, frequency, extent, duration, and delivery setting; (iii) appropriate to the adverse health condition for which it is provided and is expected to produce the desired outcome; (iv) is the lowest cost alternative that effectively addresses and treats the medical problem; (v) provides unique, essential, and appropriate information if it is used for diagnostic purposes; and (vi) not provided primarily for the economic benefit of the provider nor for the convenience of the provider or anyone else other than the recipient.

(d) The fact that a physician, dentist or other licensed practitioner renders, prescribes, orders, certifies, recommends, approves, or submits a claim for a procedure, item, or service does not, in and of itself, make the procedure, item, or service medically necessary and not guarantee payment for it.

(e) More specific criteria regarding the conditions of medical necessity for particular categories of service may set forth within Agency coverage policies or rules.

1.11 MEMBER. An individual who is eligible for the Program and who has enrolled in ACOH under the Program.

1.12 MEMBER APPEAL PROCEDURES. The written procedures describing ACOH's standards for the prompt resolution of Member problems, grievances and appeals, as described in the Provider Manual.

1.13 PARTICIPATING HOSPITAL. A duly licensed hospital which has entered into an agreement with ACOH to provide hospital-based Covered Services to Members.

1.14 PARTICIPATING PROVIDER. A physician duly licensed to practice medicine in the State of Ohio, participating in the Ohio Medicaid program, and who is a member of the medical staff of a(n) ACOH- participating hospital, or a licensed, appropriately supervised allied health professional, either of whom has entered into, or who is recognized by ACOH as a member of a group which has entered into, an agreement with ACOH to provide medical services to Members under the Program.

1.15 PRIMARY CARE PROVIDER. As set out in Ohio Administrative Code §5160-26-01(FF), an individual physician (M.D. or D.O.), a physician group practice, an advanced practice registered nurse as defined in section 4723.1 of the Ohio Revised Code, and advanced practice nurse group practice within an acceptable specialty, or a physician assistant who meets the requirements of rule 5160-4-03 of the Ohio Administrative Code contracting with ACOH to provide services as specified in OAC Rule 5160-26-03.1. Acceptable PCP specialty types include family/general practice, internal medicine, pediatrics, and obstetrics/gynecology (OB/GYN). Without limiting the applicability of OAC 5160-26-03.1, the PCP is responsible for the supervision, coordination, and provision of primary care services to Members who have selected, or have been assigned to, that provider. The Primary Care Provider also is responsible for initiating any required referrals for specialty care needed by a Member and maintaining overall continuity of a Member's care.

1.16 PRIMARY CARE SERVICES. Covered Services specified in Appendix A hereto and any additional services specified as Primary Care Services in the Provider Manual, as updated or amended from time to time.

1.17 PROGRAM. The Ohio Medicaid managed care program.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="3" -->

1.18 PROVIDER MANUAL. The ACOH manual of standards, policies, procedures and corrective actions together with amendments or modifications ACOH may adopt from time to time. The Provider Manual is herein incorporated by reference and made part of this Agreement. The Provider Manual may be amended or modified by ACOH from time to time in accordance with Section 3.4 herein below.

1.19 QUALITY MANAGEMENT PROGRAM. An ongoing review process and plan which functions to define, monitor, review, and recommend corrective action for managing and improving the quality of health care services to Members.

1.20 SPECIALTY CARE PROVIDER. A duly licensed physician who has been successfully credentialed and who has entered into an agreement to provide Specialty Care Services to Members in accordance with the referral and preauthorization requirements of the Provider Manual.

1.21 SPECIALTY CARE SERVICES. Covered Services specified in Appendix A hereto and any additional specified as "Specialty Care Services" in the Provider Manual, as updated and amended from time to time

1.22 UTILIZATION MANAGEMENT PROGRAM. A process of review of the medical necessity, appropriateness and efficiency of health care services, procedures, equipment, supplies, and facilities rendered to Members.


## 2\. OBLIGATIONS OF HEALTH SYSTEM PROVIDERS:

2.1 Throughout the term of this Agreement, Health System Providers shall have and maintain, without restriction, all licenses, certificates, registrations and permits as are required under applicable State and federal statutes and regulations to provide the Covered Services furnished by the Health System Providers and/or other related activities delegated by ACOH under this Agreement. Each Health System Provider shall obtain a unique identifier (national provider identifier) in accordance with the system established under Section 1173(b) of the Social Security Act, submit such identifier number to ACOH, and include such identifier on all claims. At all times during the term of this Agreement, all Health System Providers shall participate in the Ohio Medicaid program and be enrolled with the Agency. To the extent that Covered Services are furnished to Medicare beneficiaries under this Agreement, Health System Providers shall also participate in the Medicare program. Health System shall ensure that all services provided pursuant to this Agreement are within the applicable Health System Provider's scope of professional responsibility.

2.2 Health System shall ensure that any employed physician, or any physician practice which Health System or Health System Hospital Providers own or control that has admitting privileges at or provides services at a Health System Hospital, shall accept all Members. Health System shall notify ACOH within sixty (60) days of the acquisition of any physician practice by Health System or a Health System Hospital Provider if such practice has admitting privileges at or provides services at the Health System Hospital Provider. Health System acknowledges that if any acquired physician practice is participating with ACOH at the time of such acquisition, appropriate addenda shall be executed by Health System, the physician/physician practice, and ACOH, as may be applicable. If the physician practice or new employed physician is not participating with ACOH at the time of such acquisition or employment, as applicable, upon request by ACOH, Health System shall ensure that such physician promptly apply for ACOH participation, comply with applicable credentialing requirements and execute an appropriate ACOH provider agreement. Providers must be successfully credentialed by ACOH prior to

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="4" -->

providing services to Members; provided, however, that once Agency implements its centralized credentialing process: (i) providers must be successfully credentialed or approved through Agency's process prior to providing services to Members; and (ii) ACOH will no longer credential providers if the provider is enrolled with the Agency. In the event Health System must limit the growth of any such acquired physician practice due to capacity constraints, Health System shall cause the practice to make itself available to Members on the same basis as it is to any other patients.


### 2.3 Hospital Services.

(a) Health System Providers that are Hospitals ("Health System Hospital Providers") shall provide to Members the Covered Services described in Appendix A hereto; provided, however, that Health System Hospital Providers shall only be obligated to provide Covered Services to a Member upon an admission or referral of said Member to Health System Hospital Provider by a Participating Provider and otherwise in accordance with ACOH's admission policies as described in the Provider Manual, other than Emergency Services, which will be provided as needed. In providing Covered Services, Health System Hospital Providers agree to abide by the relevant standards, policies and procedures of ACOH, including, but not limited to administrative, credentialing, quality management, utilization management, and Member Appeal Procedures set forth in the Provider Manual and other ACOH notices. Health System Hospital Providers shall provide Covered Services in the same manner and with the same availability as services provided to other patients without regard to reimbursement and shall further provide these services in the most cost effective setting in accordance with appropriate quality of care and performance standards which are professionally recognized as industry standards and/or otherwise adopted, accepted or established by ACOH.

(b) Health System Hospital Providers' charges for Covered Services to Members shall be the same as their charges to all patients for the same services. During the term of this Agreement, Health System Hospital Providers shall maintain in their accounting records a system of recording gross charges by patient and payor. Health System shall submit its then current schedule of charges by revenue code and procedure code to ACHO upon thirty (30) days prior written request


### 2.4 Physician Services.

(a) Health System shall cause its Health System Providers who are physicians ("Health System Physician Providers") to provide, as applicable, (i) Primary Care Services to Members who have selected, or are otherwise assigned to, the Health System Physician Provider as their Primary Care Provider, and (ii) Specialty Care Services to Members who have been referred to the Health System Physician Provider. Covered Services shall be provided in accordance with the terms of this Agreement and ACOH referral, preauthorization and other Utilization Management Program polices as described in the Provider Manual, other than Emergency Services, which will be provided as needed. Health System Physician Providers will refer Members to providers participating in the ACHO network whenever Health System Physician Provider is unable to provide Medically Necessary services and/or when consistent with sound medical judgment and accepted standards of care. Health System Physician Providers shall provide such services in the same manner and with the same availability as services provided to other patients without regard to reimbursement and shall further provide these services in accordance with the clinical quality of care and performance standards which are

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="5" -->

professionally recognized as industry practice and/or otherwise adopted, accepted or established by ACOH.

(b) Health System Physician Providers will deliver office-based medical services to Members only at those office locations set forth in Appendix D hereto as such appendix is modified from time to time by mutual agreement of the parties. Health System shall notify ACOH at least sixty (60) days prior to making any addition or change to office locations.

(c) Primary Care Providers shall accept as patients those Members who have selected or have been assigned to them, and Specialty Care Providers shall accept as patients those Members who have been referred to them, in either case without regard to the health status or medical condition of such Members. Primary Care Providers may decline to accept additional Members (excluding persons already in the Primary Care Provider's practice that enroll in ACOH) by giving ACOH written notice of such intent ninety (90) days in advance of the effective date of such closure. The Primary Care Provider must accept any Members selecting the Primary Care Provider's practice during the ninety (90) day notice period.

2.5 Health System Providers shall provide ACOH with complete and accurate statements of all Covered Services provided to Members in conformance with ACOH billing procedures, including without limitation, use of complete applicable diagnosis, procedure and revenue codes, and applicable Present on Admission (POA) indicators for primary and secondary diagnoses. ACOH will not be liable for any bills relating to services that are submitted the later of: (a) after twelve (12) months from the date the services were provided or the Member was discharged (consistent with 42 CFR §447.45(d)), or (b) after sixty (60) days of the date of the Explanation of Benefits from another payor when services are first billed by the Health System Provider to another payor. Any appeal or request for adjustment of a payment by a Health System Provider must be made in accordance with applicable provisions of the Provider Manual and ACOH policies and procedures and, in any case, must be received by ACOH within the later of: (i) twelve (12) months from the date of service, or (ii) sixty (60) calendar days after the original payment, denial or partial denial; or within such other time period as may be required by the Agency. Neither Health System nor any Health System Provider may bring legal action on claims which have not been appealed through the appeal mechanisms described herein.

Encounter Data and Other Reports. Health System Providers shall deliver all reports and clinical information required to be submitted to ACOH pursuant to this Agreement for reporting purposes, including but not limited to encounter data, Healthcare Effectiveness Data and Information Set (HEDIS), Agency for Healthcare Research and Quality (AHRQ), and EPSDT data in a format which will allow ACOH to transmit required data to the Agency electronically and in a format identical to or consistent with the format used or otherwise required by ACOH and the Agency. Health System Providers shall submit this information to ACOH within the time frames set forth in the Provider Manual or as otherwise required by the Agency. Health System Providers shall submit all encounter data to the same standards of completeness and accuracy as required for proper adjudication of fee-for-service claims by ACOH.

2.6 Health System Providers may directly bill Members for non-Covered Services if the Member is advised in writing before the service is rendered: (i) the nature of the service(s) to be rendered; (ii) that ACOH does not cover the services; and (iii) that the Member will be financially responsible for the services if the Member elects to receive the services. Furthermore, Health System and Health System Providers shall hold ACOH harmless from any claim or expense arising from such services.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="6" -->

2.7 Under no circumstances, including ACOH's failure to pay for Covered Services, termination of this Agreement, or the insolvency of ACOH, will any Health System Provider make any charges or claims against any Member directly or indirectly for Covered Services authorized by ACOH, except for authorized co-payments. Health System Providers shall look only to ACOH for compensation for Covered Services. No Health System Provider shall bill or collect from any Member any amount or charges for any Covered Services provided hereunder, except for authorized co-payments, co-insurance, and/or deductibles. Health System Providers shall not deny Covered Services to a Member in the event that a Member is unable to pay any authorized co-payment amounts. 42 CFR §447.15.

2.8 During the term of this Agreement and in the event of termination of this Agreement for any reason, Health System Providers shall fully cooperate with each Member and with ACOH in arranging for the transfer of copies of Member medical records to other ACOH participating providers.

2.9 Record Maintenance, Inspection, Reporting and Auditing

(a) Record Retention. As required by 42 CFR 434.6(a)(7) and otherwise in accordance with the standards of ACOH, Health System Providers shall maintain an adequate record system for recording services, service providers, charges, dates and all other commonly required information elements for services rendered to Members pursuant to this Agreement (including but not limited to such records as are necessary for the evaluation of the quality, appropriateness, and timeliness of services performed under this Agreement and the Agency Contract).

(b) All records originated or prepared in connection with Health System Providers' performance of their obligations under this Agreement will be retained and safeguarded by the Health System Providers in accordance with the terms and conditions of the Agency Contract and other relevant State and federal law. Health System Providers shall retain all financial and programmatic records, supporting documents, statistical records and other records of Members relating to the delivery of care or service under the Agency Contract and as further required by the Agency, for a period of no less than ten (10) years from the expiration date of the Agency Contract, including any contract extension(s). If any audit, litigation, claim, or other actions involving the records have been initiated prior to the expiration of the ten (10) year period, the records shall be retained until completion of the action and resolution of all issues which arise from it or until the end of the ten (10) year period, whichever is later. If Health System Providers store records on microfilm or microfiche or other electronic means, the Provider shall produce, at its expense, legible hard copy records promptly upon the request of state or federal authorities.

(c) Medical Record Maintenance. Health System Providers shall ensure that all medical records are in compliance with the medical record keeping requirements set forth in the Provider Manual, the Agency Contract and Agency guides. Health System Providers shall maintain up-to-date medical records at the site where medical services are provided for each Member enrolled under this Agreement. Each Member's record must be accurate, legible and maintained in detail consistent with good medical and professional practice which permits effective internal and external quality review and/or medical audit and facilitates an adequate system of follow-up treatment.

(d) ACOH shall be entitled to audit, examine and inspect Health System's and Health System Providers' books and records, including but not limited to medical records, financial

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="7" -->

information and administrative information pertaining to Health System's and Health System Providers' relationship with ACOH, at any time during normal business hours, upon reasonable notice. Health System and Health System Providers shall provide ACOH, at no cost to ACOH, with such medical, financial and administrative information, and other records as may be necessary for ACOH to meet its obligations related to the Agency Contract and other regulatory obligations, Utilization Management Program and Quality Management Program standards, including NCQA standards, and other relevant accreditation standards which ACOH may require of ACOH participating providers.

2.10 Health System authorizes ACOH to include Health System Providers' name, address, telephone number, information related to Health System Providers' facilities, services and staff, and other similar information relevant to Health System Providers, their operations and staff in the ACOH provider directory and in various marketing materials identifying Health System Providers as a provider of services to Members. Health System agrees to afford ACOH the same opportunity to display brochures, signs, or advertisements in Health System's facilities as Health System affords any other insurance company or other third party payor.

2.11 While both parties support open and active communication with Members concerning Medically Necessary services, available treatment alternatives, benefit coverage information and/or any other information pertaining to the provider-patient relationship, Health System Providers shall not, during the term of this Agreement, and any renewal thereof, solicit or require any Member, either orally or in writing, to subscribe to or enroll in any managed care plan other than ACOH. The provisions of this Section 2.11 shall similarly apply to Health System's and Health System Providers' employees, agents and/or contractors.

2.12 Health System Providers shall cooperate with ACOH in the identification of other sources of payment available to Members, such as other health insurance, government programs, liability coverage, motor vehicle coverage or worker's compensation coverage, as applicable. Health System Providers shall be responsible for reporting all applicable third party resources to ACOH in a timely manner.

Health System Providers will cooperate with ACOH in coordinating benefits with other payors in accordance with coordination of benefits claim processing rules and requirements outlined in the Provider Manual, the Agency Contract, and applicable Program manuals, as amended from time to time. Health System Providers will make a reasonable attempt to determine whether any other payor has primary responsibility for the payment of a claim for services that a Health System Provider rendered to a Member and bill that payor before billing ACOH. Unless otherwise prohibited by applicable law, ACOH retains the right to recover payments made to Health System Providers in the event ACOH determines that another payor is primarily responsible for all or a portion of the claim.

2.13 Health System Providers will refer Members to ACOH-participating Health Systems whenever Health System is unable to provide Medically Necessary services and when consistent with sound medical judgment and accepted standards of care.

2.14 Health System and Health System Providers shall use best efforts to use the Agency's designated electronic utilization management and claims interfaces to improve the efficiency of utilization management and claims payment processes.

2.15 Health System and Health System Providers will assist ACOH in providing orientation services to Health System Provider staff to the extent ACOH may reasonably request.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="8" -->

2.16 Fraud and Abuse. Health System recognizes that payments made by ACOH pursuant to this Agreement are derived from federal and State funds, and acknowledges that it and/or Health System Providers may be held civilly and/or criminally liable to ACOH and/or the Agency, in the event of non-performance, misrepresentation, fraud or abuse for services rendered to Members, including but not limited to, the submission of false claims/statements for payment by Health System Providers, their employees or agents. Health System and Health System Providers shall be required to comply with all policies and procedures as developed by ACOH and the Agency, including but not limited to the requirements set forth in the Provider Manual and the Agency Contract, for the detection and prevention of fraud and abuse. Such compliance may include, but not be limited to, referral of suspected or confirmed fraud or abuse to ACOH.


## 3\. OBLIGATIONS OF ACOH:

3.1 ACOH shall pay Health System Providers for Covered Services provided to Members pursuant to the terms of this Agreement. ACOH shall have the right to offset claims payments to Health System Providers by any amount owed by the Health System Provider to ACOH, following at least thirty (30) days' written notice and the conclusion of any associated provider dispute. Health System Providers shall not be entitled to reimbursement if the Member was not eligible at the time services were rendered, and ACOH may recover any amounts paid for services rendered to an ineligible recipient following at least thirty (30) days' written notice and the conclusion of any associated provider dispute.

Overpayment Recovery. Health System Providers shall report to ACOH when they have received an overpayment, notify ACOH in writing of the reason for the overpayment, and return the overpayment to ACOH within sixty (60) calendar days after the date on which the overpayment was identified.

The Agency may recover any overpayment paid from ACOH to a Health System Provider if the Agency identifies the overpayment during the five-year period immediately following the end of the state fiscal year in which the overpayment was made, consistent with O.R.C. §5164.57.

3.2 ACOH shall compensate the applicable Health System Provider for Covered Services provided to Members upon receipt of a statement thereof, as defined in Section 2.5, and in accordance with Section 2.12 and the Covered Services Payment Schedule set forth in Appendix B but, in no event, will ACOH's payment exceed submitted charges. No additional charges will be made by Health System Providers to ACOH for Covered Services provided hereunder, and Health System Providers shall accepts the fees set forth in Appendix B as payment in full. Health System and Health System Providers understand and agree that any payments ACOH makes directly or indirectly to Health System Providers under this Agreement shall not be made as an inducement to reduce, limit or delay Medically Necessary services to any Member.

3.3 ACOH will establish payment policies for inpatient and outpatient services including, but not limited to, policies with respect to the application of claim edits, pre-admission testing, services included in inpatient rates and services included in outpatient rates. ACOH will provide at least thirty (30) days' prior written notice of any modifications to such payment policies. ACOH may, based on changes in clinical practice and modifications to standard coding systems, add and/or delete outpatient fee schedule procedures and re-categorize outpatient surgery fee schedule procedures, upon thirty (30) days' prior written notice to Health System.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="9" -->

3.4 ACOH shall furnish or otherwise make available to Health System and Health System Providers a copy of the Provider Manual, as amended from time to time. Provider Manual updates will become effective thirty (30) days from the date of notification, unless otherwise specified in writing by ACOH.

3.5 ACOH shall pay all Clean Claims for Covered Services in accordance with applicable laws, regulations and Agency requirements; and ACOH will in any event meet the claim payment timeframes required under 42 CFR §447.45(d). In its processing of claims, ACOH will apply claim edits based on sources that include CMS and state-specific policy, as set forth in the Provider Manual.

3.6 Health System Protections.

(a) ACOH shall not exclude or terminate any Health System Provider from ACOH's provider network because the Provider advocated on behalf of a Member including in the context of a utilization management appeal or another dispute with ACOH over appropriate medical care, provided that such advocacy is consistent with the degree of learning and skill ordinarily possessed by a health care provider practicing in accordance with the applicable standard of care.

(b) No Health System Provider shall be excluded or terminated from participation with ACOH due to the fact that the Provider has a practice that includes a substantial number of patients with expensive medical conditions.

(c) Health System Providers shall not be excluded from participation, nor shall this Agreement be terminated, because the Provider objects to the provision of or refuses to provide a healthcare service on moral or religious grounds.


## 4\. QUALITY MANAGEMENT/UTILIZATION MANAGEMENT:

4.1 Whether announced or unannounced, Health System and Health System Providers shall cooperate with, participate in, and abide by internal or external quality assessment reviews, Member Appeal Procedures, Utilization Management Program procedures, and Quality Management Program procedures established by ACOH and/or the Agency or their designees, and to follow practice guidelines as described in the Provider Manual, the Agency Contract, and the applicable Program manuals. Health System Providers shall permit a representative of ACOH, or its designee, to review medical records concurrently as well as retrospectively. Health System Providers shall provide copies of such medical records, either in paper or electronic form, to ACOH or its designee upon request. The Utilization Management and Quality Management Programs are described in the Provider Manual.

4.2 ACOH's Quality Management Programs consist of review of credentials and performance of hospitals and other provider types that are applying for participation in, or are participating in, ACOH's network of providers to determine whether the hospital or other provider meets ACOH's standards for quality, availability, accessibility and cooperation.

4.3 ACOH's Utilization Management Programs include requirements for pre-authorization of certain services rendered in physicians' offices and in inpatient, outpatient and ancillary Health System settings. Utilization Management Programs include concurrent, retrospective and prospective review of certain services and procedures to assure that care is delivered in the most appropriate setting and is Medically Necessary. Certain Covered Services may require prior approval from

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="10" -->

ACOH. The Covered Services subject to prior approval are more fully described in the Provider Manual and other ACOH notices. ACOH is obligated to pay for and Health System Providers are entitled to reimbursement for only those services that are Medically Necessary. Where reimbursement for an admission, inpatient day or outpatient service is denied as not prior approved or Medically Necessary, the Health System Provider shall not charge either ACOH or the Member for any health care services rendered or furnished with respect to such admission, inpatient day or outpatient service. If the Health System Provider disputes any such denial, the case in question shall be appealed through ACOH's provider appeal process. Neither Health System nor any Health System Provider may bring legal action for disputes which have not been appealed through the provider appeal process.

4.4 ACOH shall monitor and report the quality of services delivered under the Agreement and initiate a plan of correction, where necessary, to improve quality of care, in accordance with that level of care which is recognized as acceptable professional practice in the respective community in which the Health System Provider practices and/or the standards established by ACOH, the Agency, or their respective designees. Health System Providers shall cooperate with and abide by any corrective action plan initiated by ACOH and/or required by the Agency or any other State or federal regulatory agency with governing authority over the services provided under this Agreement.

4.5 Health System agrees that to the extent penalties, fines or sanctions are assessed against ACOH by the Agency or another regulatory agency with governing authority over the services provided under this Agreement as a result of Health System's or a Health System Provider's failure to comply with their obligations under this Agreement, including but not limited to, failure or refusal to respond to ACOH's the Agency's request for medical records, credentialing information, and other information required to be provided under this Agreement, Health System or the Health System Provider shall be responsible for the immediate payment of such penalties, fines or sanctions. In the event such payment is not made in a timely manner to ACOH, ACOH shall have the right to offset claims payments to the Health System Provider by the amount owed by the Health System Provider to ACOH.


## 5\. PROFESSIONAL LIABILITY INSURANCE/ADVERSE ACTIONS:

5.1 Health System, at its sole expense, shall provide professional liability, comprehensive general liability, and, as applicable, medical malpractice insurance coverage (including coverage for vicarious liability, if any, for the acts of employees, agents and representatives of Health System and Health System Providers) upon execution of this Agreement and at all times during the term of this Agreement, as follows:

(a) Amounts and extent of such insurance coverage as deemed necessary by ACOH to adequately insure Members and ACOH against any claim or claims for damages arising by reason of personal injury or death occasioned, directly or indirectly, in connection with Health System Providers' performance of any service pursuant to this Agreement; in no event shall such coverage be less than the amounts required by law.

(b) Health System shall provide ACOH with written verification of the existence of such coverage upon execution of this Agreement and as otherwise requested by ACOH throughout the term of the Agreement, which may include providing copies of face sheets of such coverage. Health System shall notify ACOH reasonably in advance of any change or cancellation of such coverage.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="11" -->

5.2 Health System shall immediately notify ACOH in writing, by certified mail, of any written or oral notice of any adverse action, including, without limitation, litigation, investigation, complaint, claim or transaction, regulatory action or proposed regulatory action, or other action naming or otherwise involving Health System, a Health System Provider or ACOH, or any other event, occurrence or situation which may reasonably be considered to have a material impact on Health System's or a Health System Providers' ability to perform their respective duties or obligations under this Agreement. Health System also shall immediately notify ACOH of any action against any applicable license, certification or participation under Title XVIII or other applicable provision of the Social Security Act or other State or federal law, State and/or DEA narcotic registration certificate, or medical staff privileges at any facility, and of any material change in the ownership or business operations of Health System or a Health System Provider. All notices required by this Section 5.2 shall be furnished as provided in Section 10.6 of this Agreement.

5.3 Health System agrees to defend, indemnify and hold harmless ACOH and its officers, directors and employees from and against any and all claims, costs and liabilities (including the fees and expenses of counsel) as a result of a breach of this Agreement by Health System or a Health System Provider, the negligent or willful misconduct of Health System or a Health System Provider and/or their respective employees, agents and representatives, and from and against any death, personal injury or malpractice arising in connection with the performance of any services by any Health System Provider in connection with this Agreement. This section shall survive the termination or expiration of this Agreement for any reason.

ACOH agrees to defend, indemnify and hold harmless Health System, Health System Providers and their respective officers, directors and employees from and against all claims, costs and liabilities (including the fees and expenses of counsel) as a result of ACOH's breach of this Agreement or the negligent or willful misconduct of ACOH and/or ACOH's employees, agents and representatives in connection with ACOH's performance under this Agreement. This section shall survive the termination or expiration of this Agreement for any reason.


## 6\. CONFIDENTIALITY:

ACOH, Health System and Health System Providers shall each comply with all applicable State and federal laws respecting the confidentiality of the medical, personal or business affairs of Members acquired in the course of providing services pursuant to this Agreement. Each party shall maintain as confidential and shall not disclose to third parties financial, operating, proprietary or business information relating to the other party which is not otherwise public information. The payment rates in this Agreement are confidential and proprietary and shall not be disclosed by either party. However, nothing herein shall prohibit either party from making any disclosure or transmission of information to the extent that such disclosure or transmission is required by the Agency or other applicable state regulatory agency, or is necessary or appropriate to enable the disclosing party to perform its obligations or enforce its rights under this Agreement, or is required by law or legal process. Should disclosure be required by law or legal process, the disclosing party shall immediately notify the other party of the disclosure.


## 7\. COOPERATION; RESOLUTION OF DISPUTES:

7.1 Cooperation. To the extent compatible with separate and independent management of each, ACOH and Health System shall at all times maintain an effective liaison and close cooperation with each other to provide maximum benefits to Members at the most reasonable cost consistent with high standards of care. ACOH, Health System and Health System Providers shall use best efforts to exchange information regarding material matters directly or indirectly related to this Agreement.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="12" -->

7.2 Resolution of Disputes. ACOH and Health System shall both fully cooperate in resolving any and all controversies among or between said parties, their employees, agents, or representatives pertaining to their respective duties under this Agreement. Such disputes shall be submitted for resolution in accordance with the provider appeal procedures as referenced in the Provider Manual and ACOH policies and procedures. Neither ACOH nor Health System or any Health System Provider shall permit a dispute between the parties to disrupt or interfere with the provision of services to Members.


## 8\. TERM; TERMINATION:

The term of this Agreement shall commence as of the Effective Date and continue for an initial one (1) year term (the "Initial Term"). After the Initial Term, the Agreement shall automatically renew for successive one (1) year terms unless the Agreement is terminated pursuant to this Section 8 as set forth herein.

Either party may terminate this Agreement without cause at the end of the Initial Term or at the end of the subsequent terms by providing the other party with at least one hundred twenty (120) days' prior written notice before the end of the then current term. The effective date of termination without cause will be on the first of the month following the expiration of the notice period. Either party may terminate this Agreement for cause due to a material breach by giving ninety (90) days' prior written notice. The notice of termination for cause will not be effective if the breaching party cures the breach within the first sixty (60) days of the ninety (90) day notice period. In the event that the breaching party does not cure the breach within the sixty (60) day period, the effective date of termination will be the first of the month following the expiration of the ninety (90) day notice period.

In the event any change in federal or State laws, rules and regulations or the Ohio Medicaid Program or the Medicare Advantage program would have a material adverse impact on either ACOH or Health System or Health System Providers in connection with the performance of this Agreement (the "Mandated Changes") such that the basis for the financial bargain of this Agreement is undermined, then the affected party shall have the right to require the other, by written notice, to enter into negotiations regarding the affected or pertinent terms of this Agreement while still maintaining the original Agreement purposes. If renegotiated, such terms shall become effective no later than thirty (30) days after the parties have reached agreement on the renegotiated terms. The parties agree to make a good faith attempt to renegotiate the Agreement to the extent necessary to comply with any Mandated Changes. If, after good faith renegotiations, the parties fail to reach an agreement satisfactory to both parties within thirty (30) days of the request for renegotiation, the party requesting such renegotiation may terminate this Agreement upon ninety (90) days prior written notice to the other party.

Notwithstanding the above, ACOH may terminate this Agreement immediately in the event any of the following occur:

8.1 If Health System or a Health System Provider is expelled, disciplined, barred from participation in, or suspended from receiving payment under any state's Medicaid program, Children's Health Insurance Program (CHIP), the Medicare Program or any other federal health care program.

8.2 If Health System or a Health System Provider is debarred, suspended or otherwise excluded from procurement or non-procurement activities under the Federal Acquisition Regulations.

8.3 Upon the loss or suspension of the Health System's liability coverage set forth under Section 5 of this Agreement.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="13" -->

8.4 With respect to a Health System Hospital Provider, the suspension or revocation of such Hospital's license or other certification or authorization, including the Hospital's JCAHO accreditation, necessary for the Hospital to render Covered Services.

8.5 Upon ACOH's reasonable determination that the health, safety or welfare of any Member may be in jeopardy if this Agreement is not terminated.

Upon termination of this Agreement for any reason, ACOH shall notify affected Members of the termination of Health System and the affected Health System Providers prior to the effective date of termination. Regardless of the reason for termination, Health System Providers shall promptly supply to ACOH all information necessary for the reimbursement of outstanding claims. 42 CFR 434.6(a)(6).


## 9\. REGULATORY AND PROGRAM-SPECIFIC PROVISIONS:

Attached hereto and incorporated herein by reference is Schedule 9, setting forth such terms and conditions as are necessary to meet State and Federal statutory and regulatory requirements, and other Agency requirements, of the Program. Schedule 9 is consecutively sub-numbered as necessary for each Program under which Health System Providers are furnishing services under this Agreement. Health System acknowledges that the specific terms as set forth in Schedule 9 are subject to amendment in accordance with federal and/or State statutory and regulatory changes to the Program. Such amendment shall not require the consent of the Health System or ACOH and will be effective immediately on the effective date thereof, as set forth in Section 10.3. In the event of a conflict between the terms of this Provider Agreement and the requirements set forth in Schedule 9, Schedule 9 shall control.


## 10\. MISCELLANEOUS:

10.1 It is understood that Health System and Health System Providers are independent contractors and in no way is Health System or any Health System Provider to be considered an employee, agent, or representative of ACOH. It is further understood that Health System Providers provide specified services to Members in exchange for an agreed upon fee. This Agreement shall not create, nor be deemed or construed to create any relationship between ACOH and Health System or any Health System Provider other than that of independent contractors, contracting with each other solely for the purpose of performing this Agreement. Each party shall be liable solely for their own activities and neither ACOH nor Health System or any Health System Provider shall be liable to any third party for the activities of the other party to this Agreement.

10.2 This Agreement, being for the purpose of retaining the professional services of Health System Providers, shall not be assigned, subcontracted, or delegated by Health System without the express written consent of ACOH.

10.3 No alterations or modifications of the terms of this Agreement shall be valid unless such alterations or modifications are incorporated into the Agreement through a written amendment, signed by both parties hereto, and attached to this Agreement; provided, however, ACOH may amend this Agreement with 30 days' notice to Health System via a(n) ACOH bulletin or other written communication provided in accordance with the notice provisions in Section 10.6, and unless Health System notifies ACOH, as applicable, of any objection, such amendment shall then take effect. Any amendment to this Agreement subject to prior regulatory approval(s) shall be effective once such regulatory approval(s) has been received.

Notwithstanding the foregoing, amendments required because of legislative, regulatory or governmental agency requirements do not require the consent of Health System or ACOH and shall

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="14" -->

be effective immediately on the effective date thereof. This Agreement remains subject to the approval of the State of Ohio, and may be amended by ACOH to comply with any requirements of the State of Ohio. Health System acknowledges that all Agency requirements, as may be amended from time to time, are incorporated to this Agreement.

10.4 This Agreement shall be deemed to have been made and shall be construed and interpreted in accordance with the laws of the State of Ohio.

10.5 This Agreement and its exhibits, appendices, schedules, addenda or other attachments constitute the entire understanding and agreement between the parties concerning the subject matter hereof. This Agreement supersedes all prior written or oral agreements or understandings existing between the parties concerning the subject matter hereof including, but not limited to, any such agreement which may have been previously executed between Health System or any Health System Provider and ACOH or any of its Affiliates relating to the provision of Covered Services under the Program. In the event of a conflict between the terms of this Agreement and the Provider Manual, the terms of the later document shall control.

10.6 Written notices to be given hereunder shall be sent by Certified Mail, Return Receipt Requested, or by an overnight delivery service which provides a written receipt evidencing delivery to the address set forth by the party, or by confirmed facsimile followed by written notice through the U.S. postal service. All notices called for hereunder shall be effective upon receipt.

If to Health System:

With a copy to:

If to AmeriHealth Caritas Ohio, Inc .:

With a copy to: General Counsel AmeriHealth Caritas 200 Stevens Drive Philadelphia, PA 19113

10.7 Both parties agree that there shall be no discrimination in the performance of this Agreement against any patient or other person as the result of that individual's race, color, religion, gender, sexual orientation, handicap, age, national origin, source of payment, or any other basis prohibited by law.

10.8 The failure of any of the parties to insist upon strict performance of any of the terms of this Agreement shall not be deemed a waiver of any of their respective rights or remedies, and shall not be deemed a waiver of any subsequent breach or default in any of the terms contained in this Agreement.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="15" -->

10.9 In the event that any provision under this Agreement is declared null or void, for any reason, the remaining provisions of this Agreement shall remain in full force and effect.

10.10 The parties will use reasonable care and due diligence in performing this Agreement. Health System Providers will be solely responsible for the services provided under this Agreement.

10.11 All captions contained in this Agreement are solely for the convenience of the parties hereto and shall not be deemed part of the content of this Agreement.

10.12 All terms used in this Agreement are deemed to refer to the masculine, feminine, neuter, singular or plural as the content may require.

10.13 Non-Discrimination. Health System and Health System Providers shall comply with (i) Title VI of the Civil Rights Act of 1964 and the rules, regulations, and order; (ii) the Rehabilitation Act of 1973 and the rules, regulations, and orders thereunder; (iii) the Americans With Disabilities Act of 1990 and the rules, regulations, and orders thereunder; and (iv) any and all applicable laws, rules and regulations prohibiting discriminatory practices. Furthermore, in accordance with Title VI of the Civil Rights Act of 1964 and the rules, regulations and orders thereunder, Health System and Health System Providers shall take adequate steps to ensure that Members with limited English skills receive free of charge the language assistance necessary to afford them meaningful and equal access to the benefits and services provided under this Agreement (see 42 U.S.C. 2000d et seq. and 45 C.F.R. Part 80, 2001 as amended).

10.14 No Offshore Contracting. No Covered Services under this Agreement may be performed outside of the United States without ACOH's prior written consent. In addition, neither Health System nor any Health System Provider will hire any individual to perform any services under this Agreement if that individual is required to have a work visa approved by the U.S. Department of Homeland Security and such individual has not met this requirement.

[SIGNATURES ON FOLLOWING PAGE; REMAINDER OF PAGE INTENTIONALLY BLANK]

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="16" -->

IN WITNESS WHEREOF, and intending to be legally bound hereby, the parties hereto, each by its officers duly authorized, hereby affix their hands as of the date written below.

| HEALTH SYSTEM | AmeriHealth Caritas Ohio, Inc. |
| - | - |
| Print Name | Name |
| Signature | Signature |
| Title | Title |
| Address | Date Effective Date of Agreement: [To be completed by AmeriHealth Caritas Ohio, Inc.] |
| National Provider ID Number ||
| Medicaid ID Number Medicare ID Number ||
| Tax ID Number | |
| Date Assignment of Payment (applicable to Group Practitioner only): Check and initial if Assignment of Payment Not Applicable: Health System Initials :unselected: | |
| By signing below, Group Physician hereby assigns and transfers all Group Physician's right to and interest in compensation payable by ACOH pursuant to this Agreement to the party identified below, and Group Physician therefore directs ACOH to pay such compensation to said party: ||
| Health System Signature | |
| Name of Group ||
| Address ||
| Group Tax ID Number ||

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="17" -->

APPENDIX A COVERED SERVICES
===


## Hospital Services

Hospital services are those healthcare services that are furnished to patients who present themselves to the hospital for treatment as inpatients, outpatients, or emergency patients. Health System Hospital Providers shall provide the following hospital services to Members:

1\. Inpatient hospital services;

2\. Emergency Services;

3\. Short procedure unit services and observation services; and

4\. Ancillary and other outpatient services.


### Physician Services

| Primary Care Services | Specialty Care Services |
| - | - |
| In Health System Physician Providers' capacity as a Primary Care Provider, Health System Physician Providers shall provide all basic health services to Members who have selected or been assigned to Provider as their Primary Care Provider including the following: | In Health System Physician Providers' capacity as a Specialty Care Provider, Provider shall provide all Specialty Care Services to Members including the following: · Ambulatory care visits; |
| · All primary ambulatory care visits and routine office procedures; | · Arrange for and/or provide inpatient medical care at ACOH-participating |
| · Periodic physical examinations; | hospital providers; and · Emergency or consultative Specialty Care Services twenty-four (24) hours a day/seven (7) days a week. |
| · Routine injections and immunizations, including vaccinations; · Arrange for and/or provide inpatient medical care at ACOH-participating hospital providers; · Referrals, as required, to Specialty Care Providers; ||
| · Referrals, as required, to ACOH-participating providers for lab, radiology and other appropriate services; ||
| · Provision or arrangement for basic health services twenty-four (24) hours a day/seven (7) days a week; and | |
| · Exercise primary responsibility for arranging and coordinating the delivery of Medically Necessary health care services to Members. | |

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="18" -->

Ancillary Services Health System Providers shall furnish the following Covered Services to Members:

[Insert description of Covered Services - specific to Provider]

Health System Providers' compensation for Covered Services is set forth in Appendix B.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="19" -->

APPENDIX B COVERED SERVICES COMPENSATION SCHEDULE
===


## Hospital Services

Commencing on the Effective Date, ACOH will compensate Health System Hospital Providers for Covered Services rendered by Health System Hospital Provider to Members in accordance with the terms of this Agreement at a rate of [ ], less applicable co-insurance and deductibles. Payments will be made in accordance with Ohio Medicaid payment policies, and with retroactive effective dates as necessary to coincide with the effective dates of changes made by the Agency to its fee schedule. In no event will ACOH's payment exceed Hospital's charges.


## Primary Care Provider Compensation

Commencing on the Effective Date, ACOH will compensate Health System Physician Providers for all Primary Care Services rendered by Health System Physician Providers to Members in accordance with the terms of this Agreement at a rate of [ ], less applicable co-insurance and deductibles. Payments will be made in accordance with Ohio Medicaid payment policies, and with retroactive effective dates as necessary to coincide with the effective dates of changes made by the Agency to its fee schedule. In no event will ACOH's payment exceed Provider's charges.


### Specialty Care Provider Compensation

Commencing on the Effective Date, ACOH will compensate Health System Physician Providers for all Specialty Care Services rendered by Health System Physician Providers to Members in accordance with the terms of this Agreement at a rate of [ ], less applicable co-insurance and deductibles. Payments will be made in accordance with Ohio Medicaid payment policies, and with retroactive effective dates as necessary to coincide with the effective dates of changes made by the Agency to its fee schedule. In no event will ACOH's payment exceed Specialty Provider's charges.


#### Ancillary Services Compensation

Commencing on the Effective Date, ACOH will compensate the applicable Health System Provider for ancillary Covered Services rendered by such Provider to Members in accordance with the terms of this Agreement at a rate of [\_]. In no event will ACOH's payment exceed the Health System Provider's charges.


#### Value-Based Program Participation

By participating with ACOH as a participating provider, the Members assigned to the Health System's associated Primary Care Providers will be included in the applicable PerformPlus® value-based program(s), as determined by provider type and specialty. The provider-specific program will be implemented pursuant to a written addendum to, and will thereby become incorporated to, this Agreement. For the first year of the program, there will be no downside risk as the program will qualify as meeting state-based requirements. For ongoing participation in a PerformPlus value-based program beyond the first year, the parties agree to negotiate in good faith within one-hundred twenty (120) days of the end of the first year of this Agreement to agree to terms for a value-based model that may include an element of downside risk. In the absence of such an agreement, the PerformPlus value-based program will terminate at the conclusion of the first year of the program.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="20" -->
:unselected: :unselected: :unselected: :unselected:
APPENDIX C ACOH AFFILIATES
===

No ACOH Affiliates are covered by this Agreement.

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="21" -->

APPENDIX D
===

HEALTH SYSTEM PHYSICIAN PROVIDERS AND OFFICE LOCATIONS COVERED BY AGREEMENT
===


## PRIMARY/SPECIALTY CARE PROVIDER(S)


## PRIMARY/SPECIALTY CARE PROVIDER(S)

Name

Name

Name

Name

Name

Name

Name

Name

Name

Name


## PRACTICE LOCATION ADDRESS


## PRACTICE LOCATION ADDRESS

Address

Address

City, State, ZIP

City, State, ZIP

Phone Number

Phone Number

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="22" -->

Schedule 9-1 Federal Requirements - Medicaid and Medicaid Managed Care (Rev. 7/1/17)

1\. No payment will be made to Provider for provider-preventable conditions or health care-acquired conditions. For purposes hereof:

a. Health care-acquired condition ("HAC") means a condition occurring in any inpatient hospital setting, identified as a HAC by the Secretary of the U.S. Department of Health and Human Services ("HHS") under section 1886(d)(4)(D)(iv) of the Social Security Act (the "Act") for purposes of the Medicare program identified in the State plan as described in section 1886(d)(4)(D)(ii) and (iv) of the Act, other than deep vein thrombosis/pulmonary embolism as related to total knee replacement or hip replacement surgery in pediatric and obstetric patients.

b. Other provider-preventable condition means a condition occurring in any health care setting that meets the following criteria: (i) is identified in the Ohio Medicaid plan; (ii) has been found by the Ohio, based upon a review of medical literature by qualified professionals, to be reasonably preventable through the application of procedures supported by evidence-based guidelines (iii) has a negative consequence for the Member; (iv) is auditable; and (v) includes, at a minimum, wrong surgical or other invasive procedure performed on a patient, surgical or other invasive procedure performed on the wrong body part, or surgical or other invasive procedure performed on the wrong patient.

c. Provider-preventable condition ("PPC") means a condition that meets the definition of "health care-acquired condition" or an "other provider-preventable condition."

No reduction in payment will be made for a PPC when the condition existed prior to the initiative of treatment for that patient by Provider. Provider shall identify PPCs when submitting claims for payment or, if no claim will be submitted, if Medicaid payment would otherwise be available for the course of treatment in which the PPC occurred, or as otherwise required by the State. 42 CFR §§438.3(g), 434.6(a)(12) and 447.26.

2\. Physician Incentives. Provider shall disclose to ACOH annually any Physician Incentive Plan (PIP) or risk arrangements Provider may have with physicians, either within Provider's group practice or other physicians not associated with Provider's group practice, even if there is no substantial financial risk between ACOH and the physician or physician group. The term "substantial financial risk" means a financial risk set at greater than twenty-five percent (25%) of potential payments for Covered Services, regardless of the frequency of assessment (i.e., collection) or distribution of payments. The term "potential payments" means simply the maximum anticipated total payments that the physician or physician group could receive if the use or cost of referral services were significantly low. 42 CFR §§438.3(i), 422.208, 422.210.

3\. Provider Discrimination Prohibited. ACOH may not, with respect to Provider participation, compensation or indemnification under this Agreement, discriminate against Provider to the extent that the Provider is acting within the scope of his, her or its license or certification under applicable State law, solely on the basis of that license or certification. Without limiting the foregoing, ACOH shall not discriminate against Provider for serving high-risk populations or specializing in conditions that require costly treatment. Nothing herein shall be construed to: (i) require ACOH to contract with Provider if not necessary to meet the needs of Members; (ii) preclude ACOH from using different reimbursement amounts for different specialties or for different practitioners in the same specialty; or (iii) preclude ACOH from establishing measures

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="23" -->

that are designed to maintain quality of services and control costs and are consistent with ACOH's responsibilities to Members. 42 CFR §§438.12, 438.214(c).

4\. Member Rights. Provider shall adhere to all applicable Federal and State laws that pertain to Member rights, and shall take such rights into account when furnishing services to Members. 42 CFR §438.100(a)(2).

5\. Provider-Member Communications. Nothing in this Agreement shall be construed to prohibit, restrict or impede Provider's ability to freely and openly discuss with Members, within the Provider's lawful scope of practice, all available treatment options and any information the Member may need in order to decide among all relevant treatment options, including but not limited to the risks, benefits and consequences of treatment or non-treatment, regardless of whether the services may be considered Covered Services in accordance with this Agreement. Further, nothing in this Agreement shall be construed to prohibit, restrict or impede Provider from discussing Medically Necessary care and advising or advocating appropriate medical care with or on behalf of a Member, including: information regarding the nature of treatment options, risks of treatment, alternative treatments or the availability of alternative therapies, consultation or tests that may be self-administered, and the Member's right to participate in decisions regarding his or her care, including the right to refuse treatment and to express preferences about future treatment decisions. 42 CFR §438.102(a).

6\. Member Hold Harmless. Provider shall accept the final payment made by ACOH as payment in full for Covered Services provided pursuant to this Agreement. Provider agrees that in no event, including, but not limited to, nonpayment by the Agency to ACOH, nonpayment by ACOH to Provider, the insolvency of ACOH, or breach of this Agreement, shall Provider bill, charge, collect a deposit from, seek compensation, remuneration or reimbursement from, solicit or accept any surety or guarantee of payment, or have any recourse against Members or persons other than ACOH acting on their behalf (including parent(s), guardian, spouse or any other person legally, or potentially legally, responsible person of the Member) for Covered Services listed in this Agreement. This provision shall not prohibit collection of supplemental charges or co-payments on ACOH's behalf made in accordance with terms of an enrollment agreement between ACOH and Members.

Provider further agrees that:

a. this hold harmless provision shall survive the termination of this Agreement regardless of the cause giving rise to termination and shall be construed to be for the benefit of Members; and that

b. this hold harmless provision supersedes any oral or written contrary agreement now existing or hereafter entered into between Provider and Members or persons acting on their behalf.

42 CFR §§438.106, 447.15.

7\. Coverage and Payment for Emergency Services. ACOH shall cover and pay for Emergency Services rendered by Provider and obtained when a Member had an Emergency Medical Condition, or when a representative of ACOH has instructed the Member to seek Emergency Services. 42 CFR §438.114(c)(1)(ii).

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="24" -->

8\. Timely Access. Provider shall meet Agency standards for timely access to care and services, taking into account the urgency of the need for services. Provider shall offer hours of operation to Members that are no less than the hours of operation offered to commercial enrollees or comparable to Medicaid fee-for-service, if Provider serves only Medicaid enrollees. Provider services shall be available 24 hours a day, 7 days a week, when medically necessary. Provider shall provide physical access, reasonable accommodations, and accessible equipment for Members with physical or mental disabilities. 42 CFR §438.206(c).

9\. Excluded Providers. Pursuant to 42 CFR §438.214(d), ACOH may not employ or contract with providers, or have a relationship with a person or entity that is excluded from participation in Federal health care programs under either Section 1128 or 1128A of the Act. ACOH may not knowingly have a Prohibited Relationship (defined hereinafter) with the following: (a) an entity or individual that is debarred, suspended or otherwise excluded from participating in procurement activities under the Federal Acquisition Regulation or from participating in non-procurement activities under regulations issued under Executive Order No. 12549 or under guidelines implementing Executive Order No. 12549; or (b) an individual or entity who is an affiliate, as defined in the Federal Acquisition Regulation at 48 CFR §2.101 of a person described in the subparagraph 9(a). For purposes of this paragraph 9, "Prohibited Relationship" includes a subcontractor of ACOH and a network provider or person with an employment, consulting or other arrangement with ACOH for the provision of items or services that are significant and material to ACOH's obligations under the Agency Contract. Provider shall comply with the disclosure, screening and enrollment requirements of 42 C.F.R. Part 455, Subparts B and E and, upon reasonable request, provide such information to ACOH in accordance with the requirements specified therein. 42 CFR §§438.608(b), 438.610

Provider represents and warrants that neither it, nor any of its contractors or employees who will furnish goods or services under the Agreement, directors or officers, or any person with an ownership interest in Provider of five percent (5%) or more, is or ever has been: (i) debarred, suspended or excluded from participation in Medicare, Medicaid, the State Children's Health Insurance Program (SCHIP) or any other Federal health care program; (ii) convicted of a criminal offense related to the delivery of items or services under the Medicare or Medicaid program; (iii) had any disciplinary action taken against any professional license or certification held in any state or U.S. territory, including disciplinary action, board consent order, suspension, revocation, or voluntary surrender of a license or certification; or (iv) debarred or suspended from participation in procurement or non-procurement activities by any federal agency (collectively, "Sanctioned Persons"). Provider shall screen all employees and contractors who will furnish goods or services under this Agreement to determine whether they have been excluded from participation in any Federal health care program, by searching applicable Federal and State databases (including but not limited to the OIG's LEIE and the NPDB) upon initial employment or engagement of or contracting with a contractor, employee, director or officer, and on a monthly basis thereafter.

Provider shall immediately notify ACOH upon knowledge by Provider that any of its contractors or employees who furnish goods or services under the Agreement, directors, officers or owners has become a Sanctioned Person, or is under any type of investigation which may result in their becoming a Sanctioned Person. In the event that Subcontractor cannot provide reasonably satisfactory assurance to ACOH that a Sanctioned Person will not receive payment from ACOH under this Agreement, ACOH may immediately terminate this Agreement. ACOH reserves the right to recover all amounts paid by ACOH for items or services furnished by a Sanctioned Person. Further, and without limiting Provider's indemnification obligations set forth elsewhere in this Agreement, to the extent penalties, fines or sanctions are assessed against ACOH as a result of Provider's having a relationship with a Sanctioned Person, Provider shall be responsible

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="25" -->

for the immediate payment of such penalties, fines or sanctions. In the event such payment is not made in a timely manner to ACOH, ACOH shall have the right to offset claims payments to Provider by the amount owed by Provider to ACOH.

10\. State and Federal Regulator Access. Provider acknowledges that the U.S. Department of Health and Human Services (HHS), Centers for Medicare and Medicaid Services (CMS), Office of the Inspector General, the Comptroller General, the Agency, and their designees may at any time inspect and audit any records or documents of Provider pertinent to this Agreement, including those pertaining to the quality, appropriateness and timeliness of services; and may at any time inspect the premises, physical facilities and equipment where Medicaid-related activities or work is conducted. The right to audit under this paragraph exists for ten (10) years from the final date of the Agency Contract or from the completion of any audit, whichever is later. 42 CFR §§434.6(a)(5), 438.3(h).

11\. Provider shall safeguard information about Members as required by Part 431, Subpart F of 42 CFR. 42 CFR §434.6(a)(8).

12\. Any permitted subcontracts entered into by Provider in order to carry out its obligations under this Agreement must be in writing and fulfill the requirements of 42 CFR Part 438 that are appropriate to the service or activity delegated under the subcontract, in accordance with 42 CFR §438.230. 42 CFR §§434.6(a)(11), (b), 438.3(k).

13\. Provider must retain, as applicable, the following information for a period of not less than ten (10) years:

a. Member grievance and appeal records in 42 CFR §438.416;

b. Base data used to determine capitation rates, in 42 CFR §438.5(c);

c. MLR reports in 42 CFR §438.8(k); and

d. The data, information and documentation specified in 42 CFR §§438.604 , 438.606, 438.608 and 438.610.

42 CFR §438.3(u).

14\. Provider shall maintain and share, as appropriate, an enrollee health record in accordance with professional standards. 42 CFR §438.208(b)(5).

15\. To the extent Provider conducts UM activities on behalf of ACOH, Provider's compensation under this Agreement shall not be structured so as to provide incentives for Provider to deny, limit or discontinue medically necessary services to any Member. 42 CFR §438.210(e).

16\. Delegation. The following provisions shall apply to the extent any of ACOH's activities or obligations under the Agency Contract are delegated to Provider:

a. The delegated activities and related reporting responsibilities will be specified in the Agreement or in a separate delegation contract;

b. Provider agrees to perform the delegated activities and reporting responsibilities in company with ACOH's Agency Contract obligations;

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="26" -->

c. ACOH may impose corrective actions, up to and including revocation of the delegated activities or obligations, in instances where the Agency or ACOH determine that Provider has not performed satisfactorily.

d. To the extent Provider is delegated responsibilities for coverage of services and payment of claims, Provider shall implement and maintain arrangements or procedures that are designed to detect and prevent fraud, waste and abuse that meet the requirements of 42 CFR §438.608(a).

Notwithstanding the foregoing, ACOH maintains ultimate responsibility for adhering to and otherwise fully complying with all terms and conditions of the Agency Contract. 42 CFR §438.230(b)(c).

17\. Provider agrees to comply with all applicable Medicaid laws, regulations (including applicable sub-regulatory guidance) and Agency Contract provisions. Provider agrees that:

a. The State, CMS, the HHS Inspector General, the Comptroller General, or their designees have the right to audit, evaluate and inspect any books, records, contracts, computer or other electronic system of Provider, or of any subcontractors, that pertain to any aspect of services and activities performed, or determination of amounts payable under the Agency Contract.

b. Provider will make available, for purposes of an audit, evaluation or inspection under subparagraph 17(a), its premises, physical facilities, equipment, books, records, contracts, computer or other electronic systems relating to ACOH's Members.

c. The right to audit under subparagraph 17(a) will exist through ten (10) years from the final date of the Agency Contract or from the date of completion of any audit, whichever is later.

d. If the State, CMS or the HHS Inspector General determines that there is a reasonable possibility of fraud or similar risk, the state, CMS or the HHS Inspector General may inspect, evaluate and audit Provider at any time.

42 CFR §438.230(c)(2), (3)

18\. ACOH may terminate this Agreement immediately upon notification from the Agency that Provider cannot be enrolled in the State Medicaid program, or if Provider has not enrolled in the State Medicaid Program within 120 days of the effective date of this Agreement. 42 CFR §438.602(b)(2).

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="27" -->

Schedule 9-2 Ohio Medicaid Addendum
===

[Attach ODM AmeriHealth Caritas Ohio, Inc. Medicaid Addendum]

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="28" -->


### Split the document into chunks base on markdown headers.

In [13]:
# headers_to_split_on = [y

#     ("Section", "Section Header"),
#     ("Article", "Article Header"),
#     ("Clause", "Clause Header"),
#     ("Sub-clause", "Sub-clause Header"),
#     ("WHEREAS", "Whereas Clause"),
#     ("THEREFORE", "Therefore Clause"),
#     ("NOTWITHSTANDING", "Notwithstanding Clause"),
#     ("1.", "Numbered List Level 1"),
#     ("1.1", "Numbered List Level 2"),
#     ("1.1.1", "Numbered List Level 3"),
#     ("----", "Custom Delimiter"),
#     ("***", "Custom Delimiter")
# ]

In [14]:
from langchain.text_splitter import CharacterTextSplitter
import re
from IPython.display import display, Markdown

# Define custom headers and patterns for splitting legal documents
headers_to_split_on = [
    r"^\d+\.\s",  # Matches "1. ", "2. ", etc., but not "1.1", "1.2"
    r"^[A-Z ]{2,50}$",  # Matches uppercase headers
    r"^# ",  # Matches Markdown level 1 headers
    r"^## "  # Matches Markdown level 2 headers
]

# Create a custom text splitter for legal documents
class LegalTextSplitter(CharacterTextSplitter):
    def __init__(self, headers_to_split_on, **kwargs):
        super().__init__(**kwargs)
        self.headers_to_split_on = [re.compile(pattern) for pattern in headers_to_split_on]

    def split_text(self, text):
        # Implement custom splitting logic based on headers_to_split_on
        chunks = []
        current_chunk = []
        for line in text.split('\n'):
            if any(pattern.match(line.strip()) for pattern in self.headers_to_split_on):
                if current_chunk:
                    chunks.append('\n'.join(current_chunk))
                    current_chunk = []
            current_chunk.append(line)
        if current_chunk:
            chunks.append('\n'.join(current_chunk))
        return chunks

# Example usage
# docs_string = """
# DEFINITIONS
# This section defines terms.

# OBLIGATIONS OF HEALTH SYSTEM PROVIDERS
# This section outlines the obligations.

# Hospital Services
# This section describes hospital services.

# 1. General Provisions
# This is the first numbered section.

# 2. Specific Provisions
# This is the second numbered section.

# 2.1 Subsection
# This is a subsection.

# 2.2 Another Subsection
# This is another subsection.
# """
docs_string = docs[0].page_content
# Initialize the custom text splitter
legal_text_splitter = LegalTextSplitter(headers_to_split_on=headers_to_split_on)

# Split the text
docs_result = legal_text_splitter.split_text(docs_string)

# Print the chunks
for i, chunk in enumerate(docs_result):
    print(f"Chunk {i+1}:\n{chunk}\n")

print("Length of splits: " + str(len(docs_result)))

# Display the last chunk using IPython display
display(Markdown(docs_result[-1]))

Chunk 1:
AmeriHealth Caritas Ohio, Inc. PROVIDER SERVICES AGREEMENT

with

[HEALTH SYSTEM]

AmeriHealth Caritas Ohio, Inc. PROVIDER SERVICES AGREEMENT
===

This Provider Services Agreement (the "Agreement"), dated as of the Effective Date (defined below), is made by and between AmeriHealth Caritas Ohio, Inc., a corporation organized under the laws of the State of Ohio, (hereinafter referred to as "ACOH") and [HEALTH SYSTEM NAME] ("Health System"), contracting for itself and on behalf of its following affiliated providers: (List them here), each of which is referred to as a "Provider" or "Health System Provider" and all of which are collectively referred to as the "Health System Providers" under this Agreement.

WHEREAS, ACOH is a managed care organization that is responsible for providing or arranging for the provision of health care services to its Members; and

WHEREAS, Health System has entered into an agreement with or is otherwise affiliated with the Health System Providers, and h

## PRACTICE LOCATION ADDRESS

Address

Address

City, State, ZIP

City, State, ZIP

Phone Number

Phone Number

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="22" -->

Schedule 9-1 Federal Requirements - Medicaid and Medicaid Managed Care (Rev. 7/1/17)

1\. No payment will be made to Provider for provider-preventable conditions or health care-acquired conditions. For purposes hereof:

a. Health care-acquired condition ("HAC") means a condition occurring in any inpatient hospital setting, identified as a HAC by the Secretary of the U.S. Department of Health and Human Services ("HHS") under section 1886(d)(4)(D)(iv) of the Social Security Act (the "Act") for purposes of the Medicare program identified in the State plan as described in section 1886(d)(4)(D)(ii) and (iv) of the Act, other than deep vein thrombosis/pulmonary embolism as related to total knee replacement or hip replacement surgery in pediatric and obstetric patients.

b. Other provider-preventable condition means a condition occurring in any health care setting that meets the following criteria: (i) is identified in the Ohio Medicaid plan; (ii) has been found by the Ohio, based upon a review of medical literature by qualified professionals, to be reasonably preventable through the application of procedures supported by evidence-based guidelines (iii) has a negative consequence for the Member; (iv) is auditable; and (v) includes, at a minimum, wrong surgical or other invasive procedure performed on a patient, surgical or other invasive procedure performed on the wrong body part, or surgical or other invasive procedure performed on the wrong patient.

c. Provider-preventable condition ("PPC") means a condition that meets the definition of "health care-acquired condition" or an "other provider-preventable condition."

No reduction in payment will be made for a PPC when the condition existed prior to the initiative of treatment for that patient by Provider. Provider shall identify PPCs when submitting claims for payment or, if no claim will be submitted, if Medicaid payment would otherwise be available for the course of treatment in which the PPC occurred, or as otherwise required by the State. 42 CFR §§438.3(g), 434.6(a)(12) and 447.26.

2\. Physician Incentives. Provider shall disclose to ACOH annually any Physician Incentive Plan (PIP) or risk arrangements Provider may have with physicians, either within Provider's group practice or other physicians not associated with Provider's group practice, even if there is no substantial financial risk between ACOH and the physician or physician group. The term "substantial financial risk" means a financial risk set at greater than twenty-five percent (25%) of potential payments for Covered Services, regardless of the frequency of assessment (i.e., collection) or distribution of payments. The term "potential payments" means simply the maximum anticipated total payments that the physician or physician group could receive if the use or cost of referral services were significantly low. 42 CFR §§438.3(i), 422.208, 422.210.

3\. Provider Discrimination Prohibited. ACOH may not, with respect to Provider participation, compensation or indemnification under this Agreement, discriminate against Provider to the extent that the Provider is acting within the scope of his, her or its license or certification under applicable State law, solely on the basis of that license or certification. Without limiting the foregoing, ACOH shall not discriminate against Provider for serving high-risk populations or specializing in conditions that require costly treatment. Nothing herein shall be construed to: (i) require ACOH to contract with Provider if not necessary to meet the needs of Members; (ii) preclude ACOH from using different reimbursement amounts for different specialties or for different practitioners in the same specialty; or (iii) preclude ACOH from establishing measures

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="23" -->

that are designed to maintain quality of services and control costs and are consistent with ACOH's responsibilities to Members. 42 CFR §§438.12, 438.214(c).

4\. Member Rights. Provider shall adhere to all applicable Federal and State laws that pertain to Member rights, and shall take such rights into account when furnishing services to Members. 42 CFR §438.100(a)(2).

5\. Provider-Member Communications. Nothing in this Agreement shall be construed to prohibit, restrict or impede Provider's ability to freely and openly discuss with Members, within the Provider's lawful scope of practice, all available treatment options and any information the Member may need in order to decide among all relevant treatment options, including but not limited to the risks, benefits and consequences of treatment or non-treatment, regardless of whether the services may be considered Covered Services in accordance with this Agreement. Further, nothing in this Agreement shall be construed to prohibit, restrict or impede Provider from discussing Medically Necessary care and advising or advocating appropriate medical care with or on behalf of a Member, including: information regarding the nature of treatment options, risks of treatment, alternative treatments or the availability of alternative therapies, consultation or tests that may be self-administered, and the Member's right to participate in decisions regarding his or her care, including the right to refuse treatment and to express preferences about future treatment decisions. 42 CFR §438.102(a).

6\. Member Hold Harmless. Provider shall accept the final payment made by ACOH as payment in full for Covered Services provided pursuant to this Agreement. Provider agrees that in no event, including, but not limited to, nonpayment by the Agency to ACOH, nonpayment by ACOH to Provider, the insolvency of ACOH, or breach of this Agreement, shall Provider bill, charge, collect a deposit from, seek compensation, remuneration or reimbursement from, solicit or accept any surety or guarantee of payment, or have any recourse against Members or persons other than ACOH acting on their behalf (including parent(s), guardian, spouse or any other person legally, or potentially legally, responsible person of the Member) for Covered Services listed in this Agreement. This provision shall not prohibit collection of supplemental charges or co-payments on ACOH's behalf made in accordance with terms of an enrollment agreement between ACOH and Members.

Provider further agrees that:

a. this hold harmless provision shall survive the termination of this Agreement regardless of the cause giving rise to termination and shall be construed to be for the benefit of Members; and that

b. this hold harmless provision supersedes any oral or written contrary agreement now existing or hereafter entered into between Provider and Members or persons acting on their behalf.

42 CFR §§438.106, 447.15.

7\. Coverage and Payment for Emergency Services. ACOH shall cover and pay for Emergency Services rendered by Provider and obtained when a Member had an Emergency Medical Condition, or when a representative of ACOH has instructed the Member to seek Emergency Services. 42 CFR §438.114(c)(1)(ii).

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="24" -->

8\. Timely Access. Provider shall meet Agency standards for timely access to care and services, taking into account the urgency of the need for services. Provider shall offer hours of operation to Members that are no less than the hours of operation offered to commercial enrollees or comparable to Medicaid fee-for-service, if Provider serves only Medicaid enrollees. Provider services shall be available 24 hours a day, 7 days a week, when medically necessary. Provider shall provide physical access, reasonable accommodations, and accessible equipment for Members with physical or mental disabilities. 42 CFR §438.206(c).

9\. Excluded Providers. Pursuant to 42 CFR §438.214(d), ACOH may not employ or contract with providers, or have a relationship with a person or entity that is excluded from participation in Federal health care programs under either Section 1128 or 1128A of the Act. ACOH may not knowingly have a Prohibited Relationship (defined hereinafter) with the following: (a) an entity or individual that is debarred, suspended or otherwise excluded from participating in procurement activities under the Federal Acquisition Regulation or from participating in non-procurement activities under regulations issued under Executive Order No. 12549 or under guidelines implementing Executive Order No. 12549; or (b) an individual or entity who is an affiliate, as defined in the Federal Acquisition Regulation at 48 CFR §2.101 of a person described in the subparagraph 9(a). For purposes of this paragraph 9, "Prohibited Relationship" includes a subcontractor of ACOH and a network provider or person with an employment, consulting or other arrangement with ACOH for the provision of items or services that are significant and material to ACOH's obligations under the Agency Contract. Provider shall comply with the disclosure, screening and enrollment requirements of 42 C.F.R. Part 455, Subparts B and E and, upon reasonable request, provide such information to ACOH in accordance with the requirements specified therein. 42 CFR §§438.608(b), 438.610

Provider represents and warrants that neither it, nor any of its contractors or employees who will furnish goods or services under the Agreement, directors or officers, or any person with an ownership interest in Provider of five percent (5%) or more, is or ever has been: (i) debarred, suspended or excluded from participation in Medicare, Medicaid, the State Children's Health Insurance Program (SCHIP) or any other Federal health care program; (ii) convicted of a criminal offense related to the delivery of items or services under the Medicare or Medicaid program; (iii) had any disciplinary action taken against any professional license or certification held in any state or U.S. territory, including disciplinary action, board consent order, suspension, revocation, or voluntary surrender of a license or certification; or (iv) debarred or suspended from participation in procurement or non-procurement activities by any federal agency (collectively, "Sanctioned Persons"). Provider shall screen all employees and contractors who will furnish goods or services under this Agreement to determine whether they have been excluded from participation in any Federal health care program, by searching applicable Federal and State databases (including but not limited to the OIG's LEIE and the NPDB) upon initial employment or engagement of or contracting with a contractor, employee, director or officer, and on a monthly basis thereafter.

Provider shall immediately notify ACOH upon knowledge by Provider that any of its contractors or employees who furnish goods or services under the Agreement, directors, officers or owners has become a Sanctioned Person, or is under any type of investigation which may result in their becoming a Sanctioned Person. In the event that Subcontractor cannot provide reasonably satisfactory assurance to ACOH that a Sanctioned Person will not receive payment from ACOH under this Agreement, ACOH may immediately terminate this Agreement. ACOH reserves the right to recover all amounts paid by ACOH for items or services furnished by a Sanctioned Person. Further, and without limiting Provider's indemnification obligations set forth elsewhere in this Agreement, to the extent penalties, fines or sanctions are assessed against ACOH as a result of Provider's having a relationship with a Sanctioned Person, Provider shall be responsible

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="25" -->

for the immediate payment of such penalties, fines or sanctions. In the event such payment is not made in a timely manner to ACOH, ACOH shall have the right to offset claims payments to Provider by the amount owed by Provider to ACOH.

10\. State and Federal Regulator Access. Provider acknowledges that the U.S. Department of Health and Human Services (HHS), Centers for Medicare and Medicaid Services (CMS), Office of the Inspector General, the Comptroller General, the Agency, and their designees may at any time inspect and audit any records or documents of Provider pertinent to this Agreement, including those pertaining to the quality, appropriateness and timeliness of services; and may at any time inspect the premises, physical facilities and equipment where Medicaid-related activities or work is conducted. The right to audit under this paragraph exists for ten (10) years from the final date of the Agency Contract or from the completion of any audit, whichever is later. 42 CFR §§434.6(a)(5), 438.3(h).

11\. Provider shall safeguard information about Members as required by Part 431, Subpart F of 42 CFR. 42 CFR §434.6(a)(8).

12\. Any permitted subcontracts entered into by Provider in order to carry out its obligations under this Agreement must be in writing and fulfill the requirements of 42 CFR Part 438 that are appropriate to the service or activity delegated under the subcontract, in accordance with 42 CFR §438.230. 42 CFR §§434.6(a)(11), (b), 438.3(k).

13\. Provider must retain, as applicable, the following information for a period of not less than ten (10) years:

a. Member grievance and appeal records in 42 CFR §438.416;

b. Base data used to determine capitation rates, in 42 CFR §438.5(c);

c. MLR reports in 42 CFR §438.8(k); and

d. The data, information and documentation specified in 42 CFR §§438.604 , 438.606, 438.608 and 438.610.

42 CFR §438.3(u).

14\. Provider shall maintain and share, as appropriate, an enrollee health record in accordance with professional standards. 42 CFR §438.208(b)(5).

15\. To the extent Provider conducts UM activities on behalf of ACOH, Provider's compensation under this Agreement shall not be structured so as to provide incentives for Provider to deny, limit or discontinue medically necessary services to any Member. 42 CFR §438.210(e).

16\. Delegation. The following provisions shall apply to the extent any of ACOH's activities or obligations under the Agency Contract are delegated to Provider:

a. The delegated activities and related reporting responsibilities will be specified in the Agreement or in a separate delegation contract;

b. Provider agrees to perform the delegated activities and reporting responsibilities in company with ACOH's Agency Contract obligations;

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="26" -->

c. ACOH may impose corrective actions, up to and including revocation of the delegated activities or obligations, in instances where the Agency or ACOH determine that Provider has not performed satisfactorily.

d. To the extent Provider is delegated responsibilities for coverage of services and payment of claims, Provider shall implement and maintain arrangements or procedures that are designed to detect and prevent fraud, waste and abuse that meet the requirements of 42 CFR §438.608(a).

Notwithstanding the foregoing, ACOH maintains ultimate responsibility for adhering to and otherwise fully complying with all terms and conditions of the Agency Contract. 42 CFR §438.230(b)(c).

17\. Provider agrees to comply with all applicable Medicaid laws, regulations (including applicable sub-regulatory guidance) and Agency Contract provisions. Provider agrees that:

a. The State, CMS, the HHS Inspector General, the Comptroller General, or their designees have the right to audit, evaluate and inspect any books, records, contracts, computer or other electronic system of Provider, or of any subcontractors, that pertain to any aspect of services and activities performed, or determination of amounts payable under the Agency Contract.

b. Provider will make available, for purposes of an audit, evaluation or inspection under subparagraph 17(a), its premises, physical facilities, equipment, books, records, contracts, computer or other electronic systems relating to ACOH's Members.

c. The right to audit under subparagraph 17(a) will exist through ten (10) years from the final date of the Agency Contract or from the date of completion of any audit, whichever is later.

d. If the State, CMS or the HHS Inspector General determines that there is a reasonable possibility of fraud or similar risk, the state, CMS or the HHS Inspector General may inspect, evaluate and audit Provider at any time.

42 CFR §438.230(c)(2), (3)

18\. ACOH may terminate this Agreement immediately upon notification from the Agency that Provider cannot be enrolled in the State Medicaid program, or if Provider has not enrolled in the State Medicaid Program within 120 days of the effective date of this Agreement. 42 CFR §438.602(b)(2).

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="27" -->

Schedule 9-2 Ohio Medicaid Addendum
===

[Attach ODM AmeriHealth Caritas Ohio, Inc. Medicaid Addendum]

<!-- PageFooter="ACOH Health System Agreement Rev.11.15.2021" -->

<!-- PageNumber="28" -->


### Define an LLM 

In [15]:
llm = AzureChatOpenAI(api_key = os.environ["AZURE_OPENAI_API_KEY"],  
                      api_version = "2024-06-01",
                      azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
                      model= os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
                      streaming=True)
resp = llm.invoke([HumanMessage("Hi")])
print(resp)

content='Hello! How can I assist you today?' response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_67802d9a6d'} id='run-a577f2da-3a7a-4a26-b218-c8e17efd6abf-0'


### Map Reduce Summarization 

In [16]:
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import Document
docs = [Document(page_content=t) for t in docs_result]

In [17]:
# chain = load_summarize_chain(llm, chain_type="map_reduce")
# chain.run(docs)

In [18]:
# result = chain.run(docs)
# print(result)

In [19]:
# chain = load_summarize_chain(llm, chain_type="map_reduce")
# chain.run(docs)

## The `map_reduce` Chain

This sections shows results of using the `map_reduce` Chain to do summarization.

In [20]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
m_r_text = chain.run(docs)
display(Markdown(m_r_text))

C:\Users\nag\AppData\Local\Temp\ipykernel_31536\2096206857.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  m_r_text = chain.run(docs)


The Provider Services Agreement between AmeriHealth Caritas Ohio, Inc. (ACOH) and a Health System outlines the terms for delivering healthcare services to ACOH members. The Health System, on behalf of its licensed providers, will offer medically necessary services and be compensated according to agreed terms. Key definitions and obligations include maintaining licenses, submitting accurate claims, retaining records, and adhering to ACOH's standards and policies. Providers must also comply with Medicaid regulations, avoid billing members for covered services, and participate in quality and utilization management programs. ACOH is responsible for timely payments and has rights to audit, recover overpayments, and enforce compliance. Both parties must maintain confidentiality, carry adequate insurance, and cooperate in dispute resolution.

The agreement lasts one year, with automatic renewals unless terminated. Termination can occur with proper notice or immediately under specific conditions. It includes provisions for indemnity, non-assignment, and compliance with non-discrimination laws. Compensation details are specified in appendices, and the agreement is governed by Ohio law. Finally, federal requirements for Medicaid participation are detailed, ensuring compliance with provider-preventable conditions, member rights, and regulatory access. The agreement concludes with a section for formal signatures and necessary appendices.

## The `refine` Chain

This sections shows results of using the `refine` Chain to do summarization.

In [25]:
chain = load_summarize_chain(llm, chain_type="refine")

refine_text = chain.run(docs)
print(refine_text)


The Provider Services Agreement between AmeriHealth Caritas Ohio, Inc. (ACOH) and [HEALTH SYSTEM NAME] establishes a contractual relationship in which Health System Providers will deliver healthcare services to ACOH's members. ACOH, a managed care organization, will compensate these providers according to the terms outlined in the agreement. The Health System has the legal authority to enter into this agreement on behalf of its affiliated providers, who are licensed and qualified to offer the necessary healthcare services. The agreement is based on mutual promises and benefits for both parties. Key terms such as "Affiliates," "Covered Services," "Medically Necessary," and "Primary Care Provider" are defined to ensure clarity and compliance with relevant state and federal regulations. The agreement also includes provisions for quality management, utilization management, and member appeal procedures to maintain high standards of care and service efficiency.

Health System Providers are o

In [26]:
display(Markdown(refine_text))

The Provider Services Agreement between AmeriHealth Caritas Ohio, Inc. (ACOH) and [HEALTH SYSTEM NAME] establishes a contractual relationship in which Health System Providers will deliver healthcare services to ACOH's members. ACOH, a managed care organization, will compensate these providers according to the terms outlined in the agreement. The Health System has the legal authority to enter into this agreement on behalf of its affiliated providers, who are licensed and qualified to offer the necessary healthcare services. The agreement is based on mutual promises and benefits for both parties. Key terms such as "Affiliates," "Covered Services," "Medically Necessary," and "Primary Care Provider" are defined to ensure clarity and compliance with relevant state and federal regulations. The agreement also includes provisions for quality management, utilization management, and member appeal procedures to maintain high standards of care and service efficiency.

Health System Providers are obligated to maintain all required licenses and certifications, participate in Medicaid and Medicare programs, and comply with ACOH's credentialing and utilization management procedures. They must accept all members and adhere to specified admission, referral, and billing protocols. Hospitals within the Health System must provide covered services as described in the agreement and maintain consistent charges for all patients. Physicians are required to offer primary and specialty care services and refer members to ACOH-participating providers when necessary.

Providers must ensure accurate billing, submit encounter data and other reports, and maintain comprehensive records for auditing purposes. They may bill members for non-covered services only with prior written consent. Health System Providers cannot charge members directly for covered services, except for authorized co-payments. They must also cooperate in transferring medical records upon termination of the agreement and assist in identifying other sources of payment. The agreement emphasizes the prevention of fraud and abuse, requiring compliance with all relevant policies and procedures.

ACOH is responsible for compensating Health System Providers for covered services rendered to eligible members, with the right to offset claims payments by any amount owed by the providers to ACOH. Overpayments must be reported and returned within sixty days. ACOH will not reimburse for services provided to ineligible members and may recover such payments. ACOH ensures payment policies are communicated with at least thirty days' notice prior to any changes and provides a Provider Manual with updates. ACOH commits to paying clean claims promptly in accordance with federal and state regulations. Protections for Health System Providers include prohibitions against exclusion or termination for advocating on behalf of members, treating a substantial number of patients with expensive conditions, or refusing to provide services on moral or religious grounds.

Health System Providers must comply with ACOH’s Quality Management and Utilization Management Programs, which include internal and external quality assessments, member appeal procedures, and adherence to practice guidelines. Providers are required to allow ACOH representatives to review medical records and provide necessary copies upon request. The Quality Management Program involves credentialing and performance reviews to ensure providers meet ACOH’s standards. Utilization Management Programs necessitate pre-authorizations and reviews to confirm services are medically necessary. Providers cannot charge for services deemed not medically necessary or not pre-approved, and disputes must go through ACOH’s provider appeal process. Providers must also cooperate with corrective action plans and may be responsible for penalties resulting from non-compliance.

The Health System is also required to maintain professional liability, comprehensive general liability, and medical malpractice insurance to cover claims arising from personal injury or death related to the services provided. Verification of such coverage must be provided to ACOH upon execution of the agreement and as requested thereafter. Any changes or cancellations in coverage must be communicated to ACOH in advance. Furthermore, the Health System must notify ACOH of any adverse actions, including litigation or regulatory actions, that could impact their ability to fulfill their obligations under the agreement.

Both parties agree to indemnify and hold each other harmless from claims, costs, and liabilities resulting from breaches of the agreement or negligent actions. This mutual indemnification survives the termination or expiration of the agreement.

Confidentiality is a key aspect of the agreement. Both ACOH and Health System Providers must comply with applicable state and federal laws regarding the confidentiality of medical, personal, and business information acquired while providing services. Financial, operating, proprietary, or business information of either party that is not public must be kept confidential. The payment rates in the agreement are also confidential and proprietary. Disclosures required by law or necessary for the execution of the agreement must be communicated to the other party immediately.

ACOH and the Health System must maintain an effective liaison and close cooperation to provide maximum benefits to members at reasonable costs while upholding high standards of care. Both parties agree to fully cooperate in resolving disputes, following the provider appeal procedures outlined in the Provider Manual and ACOH policies. Disputes should not disrupt service provision to members.

The term of the agreement begins on the Effective Date and continues for an initial one-year term, automatically renewing for successive one-year terms unless terminated. Either party can terminate the agreement without cause with 120 days' notice before the term ends. Termination for cause requires 90 days' notice, with the opportunity to cure the breach within 60 days. Immediate termination by ACOH is possible under specific conditions, such as loss of license or liability coverage, or if member health and safety are jeopardized. Upon termination, ACOH will inform members and ensure reimbursement of outstanding claims.

Schedule 9 of the agreement sets forth terms and conditions necessary to meet state and federal regulatory requirements, and these terms are subject to amendment based on changes in law. The terms in Schedule 9 take precedence over any conflicting terms in the main agreement.

Health System and Health System Providers are recognized as independent contractors, maintaining sole liability for their activities. The agreement cannot be assigned or subcontracted without ACOH's consent. Amendments to the agreement require written consent, though changes due to regulatory requirements are effective immediately. The agreement is governed by Ohio law and constitutes the entire understanding between the parties. Non-discrimination and non-waiver provisions are included, along with requirements for due diligence and no offshore contracting without prior consent.

Hospital services are those healthcare services that are furnished to patients who present themselves to the hospital for treatment as inpatients, outpatients, or emergency patients. Health System Hospital Providers shall provide hospital services to Members, including inpatient hospital services, emergency services, short procedure unit services, observation services, and ancillary outpatient services.

Health System Physician Providers shall offer primary care services and specialty care services to Members. Primary care services include all basic health services, periodic physical examinations, routine injections, immunizations, and referrals. Specialty care services encompass ambulatory care visits, inpatient medical care arrangements, and emergency or consultative specialty care services available 24/7.

Health System Providers' compensation for Covered Services is detailed in Appendix B. Commencing on the Effective Date, ACOH will compensate Health System Hospital Providers for Covered Services rendered by Health System Hospital Provider to Members in accordance with the terms of this Agreement at a rate of [ ], less applicable co-insurance and deductibles. Payments will be made in accordance with Ohio Medicaid payment policies, and with retroactive effective dates as necessary to coincide with the effective dates of changes made by the Agency to its fee schedule. In no event will ACOH's payment exceed Hospital's charges.

Commencing on the Effective Date, ACOH will compensate Health System Physician Providers for all Primary Care Services rendered by Health System Physician Providers to Members in accordance with the terms of this Agreement at a rate of [ ], less applicable co-insurance and deductibles. Payments will be made in accordance with Ohio Medicaid payment policies, and with retroactive effective dates as necessary to coincide with the effective dates of changes made by the Agency to its fee schedule. In no event will ACOH's payment exceed Provider's charges.

Commencing on the Effective Date, ACOH will compensate Health System Physician Providers for all Specialty Care Services rendered by Health System Physician Providers to Members in accordance with the terms of this Agreement at a rate of [ ], less applicable co-insurance and deductibles. Payments will be made in accordance with Ohio Medicaid payment policies, and with retroactive effective dates as necessary to coincide with the effective dates of changes made by the Agency to its fee schedule. In no event will ACOH's payment exceed Specialty Provider's charges.

Commencing on the Effective Date, ACOH will compensate the applicable Health System Provider for ancillary Covered Services rendered by such Provider to Members in accordance with the terms of this Agreement at a rate of [ ]. In no event will ACOH's payment exceed the Health System Provider's charges.

By participating with ACOH as a participating provider, the Members assigned to the Health System's associated Primary Care Providers will be included in the applicable PerformPlus® value-based program(s), as determined by provider type and specialty. The provider-specific program will be implemented pursuant to a written addendum to, and will thereby become incorporated to, this Agreement. For the first year of the program, there will be no downside risk as the program will qualify as meeting state-based requirements. For ongoing participation in a PerformPlus value-based program beyond the first year, the parties agree to negotiate in good faith within one-hundred twenty (120) days of the end of the first year of this Agreement to agree to terms for a value-based model that may include an element of downside risk. In the absence of such an agreement, the PerformPlus value-based program will terminate at the conclusion of the first year of the program.

No ACOH Affiliates are covered by this Agreement.

Appendix D specifies the Health System Physician Providers and their office locations covered by this Agreement.

Schedule 9-1 outlines federal Medicaid requirements that must be adhered to under the agreement, including provisions related to provider-preventable conditions, physician incentives, non-discrimination, member rights, provider-member communications, member hold harmless clauses, emergency services coverage, timely access to care, and excluded providers. Provider compliance with these requirements ensures consistency with federal regulations governing Medicaid and Medicaid managed care programs.

Schedule 9-2 includes the Ohio Medicaid Addendum, which details additional state-specific requirements that complement the federal mandates outlined in Schedule 9-1. These addenda ensure that the agreement remains in full compliance with both federal and state regulations, safeguarding the interests of ACOH members and promoting high standards of care among Health System Providers.

**Intermediate Steps**

We can also return the intermediate steps for `map_reduce` chains, should we want to inspect them. This is done with the `return_map_steps` variable.

In [22]:
chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=True)

In [23]:
result = chain.invoke({"input_documents": docs}, return_only_outputs=True)

# Extract and print intermediate steps
intermediate_steps = result.get('intermediate_steps', [])
print("Intermediate Steps:")
for step in intermediate_steps:
    print('-----------------')
    display(Markdown(step))


Intermediate Steps:
AmeriHealth Caritas Ohio, Inc. (ACOH) and [Health System Name] have entered into a Provider Services Agreement. Under this agreement, Health System and its affiliated providers (collectively "Health System Providers") will provide healthcare services to ACOH's members. ACOH, a managed care organization, will compensate the Health System Providers for these services. Both parties acknowledge their authority and qualifications to enter into and fulfill the terms of this agreement.
The document outlines the definitions of key terms used in an agreement between ACOH and the Health System. These terms include:

- **Affiliates:** Corporations or organizations controlled by ACOH.
- **Agency and Agency Contract:** The State or Federal agency and its contract with ACOH for providing health services.
- **Clean Claim:** A defect-free claim for payment.
- **Covered Services:** Medically necessary services for members.
- **Emergency Medical Condition and Services:** Conditions r